In [6]:
# gensim modules
from gensim import utils
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
from __future__ import print_function
from time import time


# random shuffle
from random import shuffle

# numpy
import numpy

# classifier
from sklearn.linear_model import LogisticRegression

import logging
import sys

t0 = time()
class TaggedLineSentence(object):
    def __init__(self, sources):
        self.sources = sources

        flipped = {}

        # make sure that keys are unique
        for key, value in sources.items():
            if value not in flipped:
                flipped[value] = [key]
            else:
                raise Exception('Non-unique prefix encountered')

    def __iter__(self):
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                for item_no, line in enumerate(fin):
                    yield TaggedDocument(utils.to_unicode(line).split(), [prefix + '_%s' % item_no])

    def to_array(self):
        self.sentences = []
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                for item_no, line in enumerate(fin):
                    self.sentences.append(TaggedDocument(utils.to_unicode(line).split(), [prefix + '_%s' % item_no]))
        return self.sentences

    def sentences_perm(self):
        shuffle(self.sentences)
        return self.sentences
        


sources = {'test-neg.txt':'TRAIN_NEG1', 'test-pos.txt':'TRAIN_POS1', 'train-neg.txt':'TRAIN_NEG', 'train-pos.txt':'TRAIN_POS', 'train-unsup.txt':'TRAIN_UNS', 'polarity_test_neg_list_file.txt':'TEST_NEG', 'polarity_test_pos_list_file.txt':'TEST_POS'}


sentences = TaggedLineSentence(sources)


model = Doc2Vec(min_count=1, window=10, size=100, sample=1e-4, negative=5, workers=7)
model.build_vocab(sentences.to_array())

for epoch in range(10):
    model.train(sentences.sentences_perm())

model.save('./imdb1231.d2v')
model = Doc2Vec.load('./imdb1231.d2v')


print("done in %0.3fs." % (time() - t0)) 


KeyboardInterrupt: 

In [ ]:
t0 = time()
train_arrays = numpy.zeros((50000, 100))
train_labels = numpy.zeros(50000)

for i in range(12500):
    prefix_train_pos = 'TRAIN_POS_' + str(i)
    prefix_train_neg = 'TRAIN_NEG_' + str(i)
    train_arrays[i] = model.docvecs[prefix_train_pos]
    train_arrays[12500 + i] = model.docvecs[prefix_train_neg]
    train_labels[i] = 1
    train_labels[12500 + i] = 0
    
for i in range(12500):
    prefix_train_pos = 'TRAIN_POS1_' + str(i)
    prefix_train_neg = 'TRAIN_NEG1_' + str(i)
    train_arrays[25000 + i] = model.docvecs[prefix_train_pos]
    train_arrays[37500 + i] = model.docvecs[prefix_train_neg]
    train_labels[25000 + i] = 1
    train_labels[37500 + i] = 0

print(train_labels)

test_arrays = numpy.zeros((2000, 100))
test_labels = numpy.zeros(2000)

for i in range(1000):
    prefix_test_pos = 'TEST_POS_' + str(i)
    prefix_test_neg = 'TEST_NEG_' + str(i)
    test_arrays[i] = model.docvecs[prefix_test_pos]
    test_arrays[1000 + i] = model.docvecs[prefix_test_neg]
    test_labels[i] = 1
    test_labels[1000 + i] = 0


print("done in %0.3fs." % (time() - t0)) 

In [ ]:
t0 = time()
classifier = LogisticRegression()
classifier.fit(train_arrays, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, penalty='l2', random_state=None, tol=0.0001)

print(classifier.score(test_arrays, test_labels))
print("done in %0.3fs." % (time() - t0)) 

In [18]:
model.most_similar('good')
model.similarity('best','great')

0.76160707097287073